<a href="https://colab.research.google.com/github/dlz256/Lab-Projects/blob/main/assn_start.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<center>

## ENSE 350 – Math Programming for Software Engineers - Laboratory

# Lab 3: RSA Cryptosystem

### University of Regina
### Faculty of Engineering and Applied Science - Software Systems Engineering
### Lab Instructor: [Adam Tilson](mailto:Adam.Tilson@uregina.ca)

</center>

### Your Name: Dillan Zurowski
### SID: 200431334

### Instructions

Complete the following cells to implement the RSA algorithm. The use of the `pow` function and `**` operator are prohibited.

### Part 1: Utility Functions

In this part of the notebook we will make some utility functions which can be used by any part of the application.

In [95]:
def gcd(a, b):
    ### Paste your code from lab 1 here!
    """ Finds the greatest common divisor of a and b
    >>> gcd(270, 192)
    6
    """
    if b == 0:
     return a
    return gcd(b, a%b)
    pass
    
def pulverizer(a, b):
    # paste your code from lab 1 here!
    """Return the gcd of a and b, an exact integer >= 0, 
    and s and t, coefficients to a and b which satisfy the equation:
    gcd(a,b) = sa + tb
    >>> pulverizer(899,  493)
    (29, -6, 11)
    """
    s1, t1, s2, t2 = 1,0,0,1 #hardcode first two rows
    run = True  #loop
    while (run):
        quot = a//b #get quot rounded down
        quot = int(quot)
        rem = a%b       #get remainder    
        s3=(s1-(quot*s2))   #function for finding s and t
        t3 = (t1-(quot*t2))
        if rem ==0:      #if at end
          return (b, s2, t2)
          run = False   #stop loop
        else:           #otherwise move down a row
          s1, t1 = s2, t2
          s2, t2 = s3, t3
          a=b
          b=rem
    pass

def compute_power_using_repeated_squaring (a, b, m):
    """Returns the reult of the power of some integer a, 
    raised to the power of integer b, modulo integer m,
    using the repeated squaring method.
    """
    if (b == 0):
        return 1;
    if (b == 1):
        return a % m;
     
    t = compute_power_using_repeated_squaring(a, int(b // 2), m);
    t = (t * t) % m;
     
    # if exponent is
    # even value
    if (b % 2 == 0):
        return t;
         
    # if exponent is
    # odd value
    else:
        return ((a % m) * t) % m;

def letter_to_ascii (letter):
    return ord(letter)

def ascii_to_letter (ascii_code):
    return chr(ascii_code)

def string_to_ascii_list(message):
    """ Freebie! Converts a string to a list of Ascii values.
    >>> string_to_ascii_list("hi")
    [104, 105]
    """
    ascii_list = []
    for letter in message:
        ascii_val = letter_to_ascii(letter)
        ascii_list.append(ascii_val)
    return ascii_list

def ascii_list_to_string(ascii_list):
    """ Freebie! Converts a list of Ascii values to a string.
    >>> ascii_list_to_string([104, 105])
    'hi'
    """
    message = ""
    for ascii_val in ascii_list:
        letter = ascii_to_letter(ascii_val)
        message += letter
    return message

import doctest
doctest.testmod()

TestResults(failed=0, attempted=14)

In [96]:
compute_power_using_repeated_squaring(5, 100000, 1000000007)

754573817

### Part 2: RSA Key Generator

The next class will generate the public and private keys. The constructor will have three variables passed in, p, q and e. The constructor should verify these values are legal, throwing an error if not. The class must, at the least, store the public and private keys, stored as tuples, eg. `(13, 133)` and `(25, 133)`. However, you may wish to store any of the following variables:

|variable | purpose |
| ---:|:---|
| `p` | A prime number used in key generation. Given. Must be kept secret! |
| `q` | A prime number used in key generation. Given. Must be kept secret! |
| `n` | Computed by $ n = p \times q $. The modulus value used in encryption and decryption. Part of the public key and private key.  |
| `phi_n` | Computed by $ \phi(n) = (p-1) \times (q-1) $. The number of relatively prime numbers to `n`. Used to ensure a legal encryption key is selected, and compute a decryption key. |
| `e` | The encryption key. Given. Must be relatively prime to `phi_n`. Part of the public key.|
| `d` | The decryption key. Computed. Part of the private key. Must be kept secret! |
| `public_key` | Everything needed to encrypt a message. May be distributed widely. |
| `private_key` | Everything needed to decrypt a message. Must be kept secret! |

In [97]:
class RSA_key_generator:
    """Generates RSA Public and Private Keys.
    Throws errors if the supplied parameters are illegal.
    
    >>> rsa_fail = RSA_key_generator(5,6,10)
    Traceback (most recent call last):
        ...
    ValueError: e and phi_n must be relatively prime
    
    >>> rsa = RSA_key_generator(7, 19, 17)
    >>> rsa.get_public_key()
    (17, 133)
    
    >>> rsa.get_private_key()
    (89, 133)
    
    """
    public_key = None
    private_key = None
    def __init__(self, p, q, e):

      phi_n = (p-1)*(q-1)
      n = p*q
      # print(phi_n)
      pulv = pulverizer(phi_n, e)
      t = pulv[2]
      if t <=0:
        d = phi_n + t
      else:
        d=t
      # print(d)
      # confirm e is valid, else raise a ValueError exception with the message:
      # "e and phi_n must be relatively prime"    
      if(gcd(e,phi_n)!=1):
        raise ValueError( "e and phi_n must be relatively prime")
         
        # compute the public and private key and store them as class members
      self.public_key = (e,n)
      self.private_key = (d,n)

      pass
       
    def get_public_key(self):
        """Returns the public key"""
        # your code here
        return self.public_key
        pass
    
    def get_private_key(self):
        """Returns the private key"""
        # your code here
        # note - in typical real world applications, the public key should NOT be distributed!
        return self.private_key
        pass
    
import doctest
doctest.testmod()

TestResults(failed=0, attempted=14)

### Part 3: Sender and Receiver

The sender and receiver objects represents seperate entities. The sender has only the public key, and thus may encrypt message. The receiver has only the private key, and thus may decrypt messages. Classes and object oriented programming are used in this sense to simulate a realistic RSA system.

In [98]:
class Sender:
    """Sender class should only have the public key, and do encrypt operations
    
    >>> sender = Sender((17, 133))
    
    >>> sender.encrypt_ascii_value(29)
    78
    
    >>> sender.encrypt_message("hi")
    [55, 21]
    
    """
    def __init__ (self, public_key):
        """initializes the sender with the public key"""
        # your code here
        self.public_key = public_key
        pass

    def encrypt_ascii_value(self, ascii_val):
        """encrypts a single ascii_value using the public key"""
        # your code here
        # use the `repeated squaring` method!
        return compute_power_using_repeated_squaring(ascii_val, self.public_key[0], self.public_key[1])
        pass

    def encrypt_message (self, message):
        """encrypts a string, returning a list of encrypted ascii values, using the public key"""
        # use my utility functions!
        # your code here
        l=string_to_ascii_list(message)
        encrypted_list =[]
        for element in l:
          encrypted_list.append(self.encrypt_ascii_value(element))
        return encrypted_list
        pass   

import doctest
doctest.testmod()

TestResults(failed=0, attempted=14)

In [99]:
class Receiver:
    """Receiver class should only have the private key, and do decrypt operations
    
    >>> receiver = Receiver((89, 133))
    
    >>> receiver.decrypt_ascii_value(78)
    29
    
    >>> receiver.decrypt_ascii_list([55, 21])
    'hi'

    """
    def __init__ (self, private_key):
        """initializes the receiver with the private key"""
        # your code here
        self.private_key = private_key
        pass

    def decrypt_ascii_value(self, ascii_val):
        """decrypts a single ascii_value using the private key"""
        # your code here
        # use the `repeated squaring` method!
        return compute_power_using_repeated_squaring(ascii_val, self.private_key[0], self.private_key[1])
        pass

    def decrypt_ascii_list (self, encrypted_ascii_list):
        """decrypts a list of encrypted ascii values, and returns a string, using the private key"""
        # use my utility functions!
        # your code here
        decrypted_list=[]
        for element in encrypted_ascii_list:
          decrypted_list.append(self.decrypt_ascii_value(element))
        return ascii_list_to_string(decrypted_list)
        pass   

import doctest
doctest.testmod()

TestResults(failed=0, attempted=14)

### Part 4: A bigger test

If all of the doctests have passed so far, your code works, at least for small numbers. The following cell will test the entire system to ensure that all of the components work together properly.

In [100]:
# A full test with (very slightly) bigger numbers! Too big to do by hand, but still trivial for computers to defeat.

message = "This is my super secret password: 123456"
print ("original message:")
print (message)
print ()

rsa = RSA_key_generator(1861, 2399, 257)
sender = Sender(rsa.get_public_key())
receiver = Receiver(rsa.get_private_key())
encrypted = sender.encrypt_message(message)
print ("encrypted:")
print (encrypted)
print ()

decrypted = receiver.decrypt_ascii_list(encrypted)
print ("decrypted:")
print (decrypted)

# Takes about 30 seconds on my laptop if I use the linear approach, and instantaneous with repeating squares!

original message:
This is my super secret password: 123456

encrypted:
[1478341, 2176183, 2045269, 3001567, 3087528, 2045269, 3001567, 3087528, 3344291, 1370142, 3087528, 3001567, 2171899, 3082605, 374070, 3725619, 3087528, 3001567, 374070, 54132, 3725619, 374070, 965414, 3087528, 3082605, 3926, 3001567, 3001567, 2427858, 2466881, 3725619, 3569310, 2027747, 3087528, 1809306, 84033, 376736, 1879096, 836351, 1770409]

decrypted:
This is my super secret password: 123456


In [101]:
# This is a real world example: 2048 bit RSA. It has not been defeated by brute force!

# importing the sys module
import sys
sys.setrecursionlimit(10**6)

message = "This is my super secret password: 123456"
print ("original message:")
print (message)
print ()

p = 145527551439945991296651804140084655992976566453410886888606719871650350905183299995311230017670557039829337550211982985082926519242231309723932605920252478803877038593489984109391972786665823683360417804799987200092260708640160760058267052498592889081725336775461880700739474023326182613557543061699260854491
q = 121479580527719119165122591960121516235938150016957147100233963399255883050506655239870633325782404612202815257491952032709482372814813265488032526415225463714631574798976443779868133870366907682750542634988390489327186161700661621349200162322081408942294171574155842769088241468940581678830896002995400794237

rsa = RSA_key_generator(p, q, 65537)
sender = Sender(rsa.get_public_key())
receiver = Receiver(rsa.get_private_key())

encrypted = sender.encrypt_message(message)
print ("encrypted:")
print (encrypted)
print ()

decrypted = receiver.decrypt_ascii_list(encrypted)
print ("decrypted:")
print (decrypted)

# Takes about a second on my laptop with repeating squares!

original message:
This is my super secret password: 123456

encrypted:
[...]

decrypted:
This is my super secret password: 123456
